In [10]:
import pandas as pd
import os
import json
import requests
import re
import warnings
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

In [11]:
path= os.getcwd()
path_credenciales= os.path.join(path, 'secretos', 'credentials.json')
with open(path_credenciales) as f:
    credenciales= json.load(f)
    client_id= credenciales['client_id']
    client_secret= credenciales['client_secret']

data= {
    'grant_type':'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_uri= 'https://accounts.spotify.com/api/token'
response= requests.post(token_uri, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
    print("Token valido.")
else:
    print("Error en la obtencion del token.")
    print(response.text)


client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

Token valido.


In [12]:
# Se crea el cliente que se utilizará para consumir la API.
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
# Se busca información sobre Blackpink.
result = sp.search(q='David Guetta', type='artist', limit=1)

# Se imprime el resultado.
result

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=David%20Guetta&type=artist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=David%20Guetta&type=artist',
  'offset': 0,
  'previous': None,
  'total': 820,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Cs0zKBU1kc0i8ypK3B9ai'},
    'followers': {'href': None, 'total': 27055990},
    'genres': ['edm', 'dance'],
    'href': 'https://api.spotify.com/v1/artists/1Cs0zKBU1kc0i8ypK3B9ai',
    'id': '1Cs0zKBU1kc0i8ypK3B9ai',
    'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5ebf150017ca69c8793503c2d4f',
      'height': 640,
      'width': 640},
     {'url': 'https://i.scdn.co/image/ab67616100005174f150017ca69c8793503c2d4f',
      'height': 320,
      'width': 320},
     {'url': 'https://i.scdn.co/image/ab6761610000f178f150017ca69c8793503c2d4f',
      'height': 160,
      'width': 160}],
    'name': 'David Guetta',
    'popularity': 91,
    't

In [14]:
# Se crea el cliente que se utilizará para consumir la API.
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:
# Se imprime el número de seguidores de BlackDavid Guettapink.
print('Seguidores:\n')
result['artists']['items'][0]['followers']['total']

Seguidores:



27055990

In [ ]:
# Se busca información sobre David Guetta y se obtiene su ID de artista.
davidguetta_id = result['artists']['items'][0]['id']

# Se obtienen las 10 canciones más escuchadas de Blackpink.
results = sp.artist_top_tracks(davidguetta_id)

# Se imprime el nombre de las canciones.
for track in results['tracks']:
    print(track['name'])

I'm Good (Blue)
Titanium (feat. Sia)
Beautiful People
Memories (feat. Kid Cudi)
Sexy Bitch (feat. Akon)
I Don't Wanna Wait
Without You (feat. Usher)
Where Them Girls At (feat. Nicki Minaj & Flo Rida)
Play Hard (feat. Ne-Yo & Akon)
Baby Don't Hurt Me


In [19]:
def get_tracks_by_genre(genre):
    """
    Función que obtiene información de 50 canciones de un género musical de
    Spotify y organiza la información de un Pandas dataframe.
    """

    # Se buscan 50 canciones del género especificado.
    results = sp.search(q=f'genre:{genre}', type='track', limit=50)

    # Se extraen los IDs de las canciones.
    track_ids = [track['id'] for track in results['tracks']['items']]

    # Se obtienen detalles de las canciones.
    tracks_info = sp.tracks(tracks=track_ids)

    # Se crea una lista para almacenar los datos.
    track_data = []

    # Se obtienen los datos de cada canción y se añaden a la lista.
    for track in tracks_info['tracks']:
        track_data.append({
            'genre': genre,
            'name': track['name'],
            'artists_name': ', '.join([artist['name'] for artist in track['artists']]),  # Si hay varios artistas
            'album_name': track['album']['name'],
            'album_release_date': track['album']['release_date'],
            'duration_ms': track['duration_ms'],
            'popularity': track['popularity'],
            'track_number': track['track_number'],
            'uri': track['uri']
        })

    # Se convierte la lista en un DataFrame de pandas.
    df = pd.DataFrame(track_data)

    return df

In [20]:
# Ejemplo de uso con el género 'rock'.
genre = "rock"
get_tracks_by_genre(genre)

,genre,name,artists_name,album_name,album_release_date,duration_ms,popularity,track_number,uri
0,rock,Por Mil Noches,Airbag,Libertad,2013-11-27,283884,74,3,spotify:track:5vHWh24GAsXJ7tnIu8v7UN
1,rock,Como Eran Las Cosas,Babasonicos,Mucho,2008-01-01,240520,73,5,spotify:track:61qPUnazSdkvua4wgA4L8C
2,rock,Tan Lejos,"No Te Va Gustar, Juan Casanova",El Camino Más Largo,2008-10-23,298000,72,7,spotify:track:44m9PAUZYeOEGeaTYh6NAl
3,rock,A las Nueve,No Te Va Gustar,El Calor Del Pleno Invierno,2012-10-18,207533,73,3,spotify:track:0CgpCtEPvyDizcg2p47VrU
4,rock,Ese Maldito Momento,No Te Va Gustar,El Calor Del Pleno Invierno,2012-10-18,185866,72,5,spotify:track:6MuJUSYCqA4FpMogUHzb2R
5,rock,Cicatrices,Airbag,Mentira La Verdad,2016-09-16,228541,70,7,spotify:track:23FJGRug9hlYUIBnrmkIBX
6,rock,Pasos Al Costado,Turf,Para Mi para Vos,2004-01-01,256555,69,3,spotify:track:7upAH0ezzsThyvr33YKn4u
7,rock,Cae el Sol,Airbag,Voragine,2011-05-01,281386,70,5,spotify:track:1Mqfdem6pgGVoULjSkkcc9
8,rock,El Fantasma,Árbol,GUAU!,2005-01-01,237800,69,11,spotify:track:0N361WhEqVunVl5ZBkfd70
9,rock,Nunca quise,Intoxicados,Otro Día en el Planeta Tierra,2005,260371,69,3,spotify:track:0ZAJ660VP57lLK4U7NlGOy


In [22]:
# Se crea una lista con los géneros musicales deseados.
genres_list = ['edm', 'dance', 'electronic']

# Se itera sobre esta lista utilizando sus elementos como entradas de la función creada.
tables = []

for item in tqdm(genres_list, desc = 'Creando conjunto de datos:'):
  table = get_tracks_by_genre(item)
  tables.append(table)

# Se concatenan las tablas y se imprime el resultado.
dataset = pd.concat(tables)
dataset

Creando conjunto de datos:: 100%|██████████| 3/3 [00:02<00:00,  1.06it/s]


,genre,name,artists_name,album_name,album_release_date,duration_ms,popularity,track_number,uri
0,edm,Eyes Wide Shut,"ILLENIUM, Avril Lavigne, Travis Barker",Hot Hits Bop,2024-05-18,152331,0,43,spotify:track:2u8slKJ6boOSfcQ5NP3ium
1,edm,Raw Joy,AA Sound System,Laissez faire...,2006-01-01,182640,0,4,spotify:track:6LD3dvmlPn1cvuI9mac0Vm
2,edm,Kids Love To Play - Ash Roy remix,"Ash Roy, Arnold From Mumbai",Kids Love To Play,2012-01-10,397000,0,3,spotify:track:2q9y10weZIx5GDUtT81It3
3,edm,Break Free - Original Mix,"Tempo Giusto, Gabriel Thomas, Catie Leta","Infinite Flame, Vol. 7",2023-08-24,215967,0,3,spotify:track:5hqt2CzHdAT3VvmjbUG02M
4,edm,Get Get Down - Radio Mix,"R3HAB, Addy Van Der Zwan",Get Get Down,2009-11-23,159375,0,1,spotify:track:2MUiWTV7Aw8i0RunJBNtQN
...,...,...,...,...,...,...,...,...,...
45,electronic,Don't You Worry Child - Radio Edit,"Swedish House Mafia, John Martin",Don't You Worry Child,2012-09-14,212862,83,1,spotify:track:2V65y3PX4DkRhy1djlxd9p
46,electronic,Stand by Me,Oasis,Be Here Now,1997-07-08,356600,69,4,spotify:track:7aSSsHnGQ54rbMZpXApmtU
47,electronic,Waves - Robin Schulz Radio Edit,"Mr. Probz, Robin Schulz",Waves (Robin Schulz Radio Edit),2014-02-07,208133,83,1,spotify:track:1HFfMOxCAT4GAwaPfCdmUs
48,electronic,Play Hard (feat. Ne-Yo & Akon),"David Guetta, Ne-Yo, Akon",Nothing but the Beat (Ultimate Edition),2012-12-07,201000,78,6,spotify:track:5YPMEOJ58kfl56VHxTgwx3


In [29]:
# Se descarga el CSV.
dataset.to_csv('archivo_salida.csv', index=False)